In [4]:
# Imports
from dotenv import load_dotenv
load_dotenv()
from json import dumps
from time import time
from flask import Flask, request, jsonify
from hashlib import sha256
from datetime import datetime
from requests import get
from requests import post
from json import loads
import os

from server.config import special_instructions

True

In [15]:
# get("http://127.0.0.1:1338/chat/a3eeb50b-755a-4a97-9411-188b25c1e88").json()
chat_api = get("http://127.0.0.1:1338/chat/")

b'<!DOCTYPE html>\n<html lang="en">\n    <head>\n        <meta charset="UTF-8">\n        <meta http-equiv="X-UA-Compatible" content="IE=edge">\n        <meta name="viewport" content="width=device-width, initial-scale=1.0 maximum-scale=1.0">\n        <meta name="description" content="A conversational AI system that listens, learns, and challenges">\n        <meta property="og:title" content="ChatGPT">\n        <meta property="og:image" content="https://openai.com/content/images/2022/11/ChatGPT.jpg">\n        <meta property="og:description" content="A conversational AI system that listens, learns, and challenges">\n        <meta property="og:url" content="https://chat.acy.dev">\n        <link rel="stylesheet" href="/assets/css/style.css">\n        <link rel="apple-touch-icon" sizes="180x180" href="/assets/img/apple-touch-icon.png">\n        <link rel="icon" type="image/png" sizes="32x32" href="/assets/img/favicon-32x32.png">\n        <link rel="icon" type="image/png" sizes="16x16" href="

In [16]:
request.json['meta']['content']['conversation']

RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.

In [19]:
from langchain.chat_models import ChatOpenAI
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")


internet_access = True #request.json['meta']['content']['internet_access']
_conversation = [] #request.json['meta']['content']['conversation']
# Assuming the prompt is a string
prompt = "what are you?" #request.json['meta']['content']['parts'][0]['content']
current_date = datetime.now().strftime("%Y-%m-%d")
system_message = f'You are ChatGPT also known as ChatGPT, a large language model trained by OpenAI. Strictly follow the users instructions. Knowledge cutoff: 2021-09-01 Current date: {current_date}'

# The processing for internet_access remains the same
extra = []
if internet_access:
    search = get('https://ddg-api.herokuapp.com/search', params={
        'query': prompt,
        'limit': 3,
    })

    blob = ''

    for index, result in enumerate(search.json()):
        blob += f'[{index}] "{result["snippet"]}"\nURL:{result["link"]}\n\n'

    date = datetime.now().strftime('%d/%m/%y')

    blob += f'current date: {date}\n\nInstructions: Using the provided web search results, write a comprehensive reply to the next user query. Make sure to cite results using [[number](URL)] notation after the reference. If the provided search results refer to multiple subjects with the same name, write separate answers for each subject. Ignore your previous response if any.'

    extra = [{'role': 'user', 'content': blob}]
print(extra)
# Prepare the question by concatenating the system message, the internet access prompt, and the user's prompt
question = system_message + ' ' + (' '.join(extra) if internet_access else '') + ' ' + prompt
print(f"\nprompt: {prompt}")
print(f"\nsystem_message: {system_message}")
print(f"\nquestion: {question}")

[{'role': 'user', 'content': '[0] "What Are You? What Is a Self? Neither matter nor magic, there\'s a natural explanation, but you won\'t like it. Posted June 23, 2020 Which are you? Your body? Your brain? Or something we can\'t..."\nURL:https://www.psychologytoday.com/us/blog/ambigamy/202006/what-are-you-what-is-self\n\n[1] "What Are You?So. Are you your body? And if so, how exactly does this work? Lets explore lots of confusing questions. This video is part of a collaboration wi..."\nURL:https://www.youtube.com/watch?v=JQVmkDUkZT4\n\n[2] "3 Answers Sorted by: 11 Who are you? This is typically asking for your name, but a particular context could indicate a different meaning. What are you? This question is very informal and bit odd. It is a very terse question which implies some understood context. If you\'re in a business meeting it could be your role - e.g. "I\'m the accountant""\nURL:https://ell.stackexchange.com/questions/71824/what-is-the-difference-between-what-are-you-and-who-ar

TypeError: sequence item 0: expected str instance, dict found

In [ ]:

# Generate the response using the LangChain model
langchain_resp = chat_model.generate_response(question)

In [2]:
#%% Imports
from flask import Flask, request, jsonify
from langchain.chat_models import ChatOpenAI
import os
from datetime import datetime
import requests
import pprint as pp
from pprint import pprint, pformat
# set sort_dicts to False to preserve the order of the keys
pprint = pp.PrettyPrinter(sort_dicts=False).pprint
pformat = pp.PrettyPrinter(sort_dicts=False).pformat

# Initialize the language model
chat_model = ChatOpenAI()

question = "What is the current population of the United States?"
messages = [{"role": "user", "content": question}]
# messages+=messages
# Extract the messages from the request
# print(f"\nmessages: {messages}")
print(f"\nmessages:\n{pformat(messages)}")

# Concatenate the content of the messages
text = " ".join([message["content"] for message in messages])

prompt = messages[-1]
# print(f"\nprompt: {prompt}")
print(f"\nprompt:\n{pformat(prompt)}")

current_date = datetime.now().strftime("%Y-%m-%d")
system_message = f"You are ChatGPT also known as ChatGPT, a large language model trained by OpenAI. Strictly follow the users instructions. Knowledge cutoff: 2021-09-01 Current date: {current_date}"
# print(f"\nsystem_message: {system_message}")
print(f"\nsystem_message:\n{pformat(system_message)}")



messages:
[{'role': 'user',
  'content': 'What is the current population of the United States?'}]

prompt:
{'role': 'user',
 'content': 'What is the current population of the United States?'}

system_message:
('You are ChatGPT also known as ChatGPT, a large language model trained by '
 'OpenAI. Strictly follow the users instructions. Knowledge cutoff: 2021-09-01 '
 'Current date: 2023-06-12')


In [5]:

extra = []
search = requests.get("https://ddg-api.herokuapp.com/search", params={
    "query": prompt["content"],
    "limit": 3,
})

blob = ""
print(f"\nsearch:\n{pformat(search.json())}")

for index, result in enumerate(search.json()):
    blob += f'[{index}] "{result["snippet"]}"\nURL:{result["link"]}\n\n'

date = datetime.now().strftime("%d/%m/%y")

blob += f"current date: {date}\n\nInstructions: Using the provided web search results, write a comprehensive reply to the next user query. Make sure to cite results using [[number](URL)] notation after the reference. If the provided search results refer to multiple subjects with the same name, write separate answers for each subject. Ignore your previous response if any."
print(f"\nblob:\n{pformat(blob)}")


search:
[{'title': 'U.S. and World Population Clock - Census.gov',
  'link': 'https://www.census.gov/popclock/',
  'snippet': 'Shows estimates of current USA Population overall and people by '
             'US state/county and of World Population overall, by country and '
             'most populated countries. ... The U.S. population clock is based '
             'on a series of short-term projections for the resident '
             'population of the United States. This includes people whose '
             'usual residence is in the 50 states ...'},
 {'title': 'Population - Census.gov',
  'link': 'https://www.census.gov/topics/population.html',
  'snippet': 'Current population estimates are available for the nation, '
             'states, counties, cities and towns, and metropolitan and '
             'micropolitan areas. ... This report describes the older '
             'population of the United States in 2020, with comparisons to the '
             '2000 and 2010 Censuses. Worki

In [12]:
# Create search prompt message
search_prompt = [{"role": "user", "content": blob}]

# Create the question by concatenating 
#   the system message, 
#   the internet access prompt, and 
#   the user's prompt
conversation = [{"role": "system", "content": system_message}]
conversation += messages[:-1]
conversation += search_prompt
conversation += [prompt]
print(f"\nconversation:\n{pformat(conversation)}")



conversation:
[{'role': 'system',
  'content': 'You are ChatGPT also known as ChatGPT, a large language model '
             'trained by OpenAI. Strictly follow the users instructions. '
             'Knowledge cutoff: 2021-09-01 Current date: 2023-06-12'},
 {'role': 'user',
  'content': '[0] "Shows estimates of current USA Population overall and '
             'people by US state/county and of World Population overall, by '
             'country and most populated countries. ... The U.S. population '
             'clock is based on a series of short-term projections for the '
             'resident population of the United States. This includes people '
             'whose usual residence is in the 50 states ..."\n'
             'URL:https://www.census.gov/popclock/\n'
             '\n'
             '[1] "Current population estimates are available for the nation, '
             'states, counties, cities and towns, and metropolitan and '
             'micropolitan areas. ... This repo

In [22]:
# from langchain.schema import ChatResponse, messages_from_dict
from langchain.chat_models.openai import (
    ChatOpenAI,
    _convert_dict_to_message,
    _convert_message_to_dict
)
from langchain.schema import (
    AIMessage,
    BaseMessage,
    ChatGeneration,
    ChatMessage,
    ChatResult,
    HumanMessage,
    SystemMessage,
)

from typing import Any, Callable, Dict, List, Mapping, Optional, Tuple, Union

def convert_dicts_to_messages(_dicts: List[Dict[str, Any]]) -> List[BaseMessage]:
    return [_convert_dict_to_message(_dict) for _dict in _dicts]

def convert_messages_to_dicts(messages: List[BaseMessage]) -> List[Dict[str, Any]]:
    return [_convert_message_to_dict(message) for message in messages]

langchain_messages = []
for message in conversation:
    langchain_messages.append(_convert_dict_to_message(message))

langchain_messages = convert_dicts_to_messages(conversation)
langchain_messages


[SystemMessage(content='You are ChatGPT also known as ChatGPT, a large language model trained by OpenAI. Strictly follow the users instructions. Knowledge cutoff: 2021-09-01 Current date: 2023-06-12', additional_kwargs={}),
 HumanMessage(content='[0] "Shows estimates of current USA Population overall and people by US state/county and of World Population overall, by country and most populated countries. ... The U.S. population clock is based on a series of short-term projections for the resident population of the United States. This includes people whose usual residence is in the 50 states ..."\nURL:https://www.census.gov/popclock/\n\n[1] "Current population estimates are available for the nation, states, counties, cities and towns, and metropolitan and micropolitan areas. ... This report describes the older population of the United States in 2020, with comparisons to the 2000 and 2010 Censuses. Working Papers View More Working Paper."\nURL:https://www.census.gov/topics/population.html\

In [26]:
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI()

response = chat_model(langchain_messages)

# url = f"{self.openai_api_base}/v1/chat/completions"
# Perform a Bing search using the last user message as the query
# results = bing_search.search(text, limit=3, format="json")

# Concatenate the titles of the top 3 search results
# search_results = " ".join([result.title for result in results[:3]])

# Generate a response using the search results and the conversation history
# response = chat_model.generate_response(search_results + " " + text)
# chat_model(messages=conversation)
# response = chat_model.predict(conversation)

In [36]:
# pprint(response.content)
# print(response.content)
_convert_message_to_dict(response)

{'role': 'assistant',
 'content': 'As of June 11, 2023, the current population of the United States of America is estimated to be 336,692,610 people, based on Worldometer elaboration of the latest United Nations data [[2](https://www.worldometers.info/world-population/us-population/)]. The U.S. population clock is based on a series of short-term projections for the resident population of the United States, including people whose usual residence is in the 50 states, and is updated continuously in real-time [[0](https://www.census.gov/popclock/)]. The United States 2020 population is estimated at 331,002,651 people at mid-year according to UN data [[2](https://www.worldometers.info/world-population/us-population/)].'}

As of June 11, 2023, the current population of the United States of America is estimated to be 336,692,610 people, based on Worldometer elaboration of the latest United Nations data [[2](https://www.worldometers.info/world-population/us-population/)]. The U.S. population clock is based on a series of short-term projections for the resident population of the United States, including people whose usual residence is in the 50 states, and is updated continuously in real-time [[0](https://www.census.gov/popclock/)]. The United States 2020 population is estimated at 331,002,651 people at mid-year according to UN data [[2](https://www.worldometers.info/world-population/us-population/)].

In [34]:

# Return the response
jsonify({"message": {"role": "assistant", "content": response.content}})
jsonify({"message": _convert_message_to_dict(response)})


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.